In [1]:
from datetime import datetime, date
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from random import random
import time
import re
# current version of seaborn generates a bunch of warnings that we'll ignore
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Use this to test for captcha block or IP ban
def get_URL(position,location):
    #from torrequest import TorRequest
    """[Build a template url for a dummy call to verify the site isn't returning a captcha]
    Args:
        position ([string]): [job for query]
        location ([string]): [location for query]
    Returns:
        [string]: [formatted url]
    """
    template = 'https://www.indeed.com/jobs?q={}&l={}&fromage=1&sort=date'
                
    position = position.replace(' ', '%20')
    location = location.replace(' ', '+')
    url = template.format(position,location)
    return url


# from torrequest import TorRequest
# tr=TorRequest(password='your_super_secure_password')
position = 'data scientist'
location = 'california'
# tr.reset_identity()
response = requests.get(get_URL(position,location))
# This will either return an HTML block for a captcha or of a search result
response

In [4]:

soup = BeautifulSoup(response.text, 'html.parser')
#soup


In [19]:
def get_desc_features(job_url):
    """Parses each job description, searching for and extracting values for features

    Args:
        job_url (string): http address of each job posting

    Returns:
        tuple: job feature values
    """
    response_job_desc = requests.get(job_url)
    soup = BeautifulSoup(response_job_desc.text, 'html.parser')

    
    try:
        salary_and_jType = soup.find('div', id='salaryInfoAndJobType').text.strip()
    except:
        salary_and_jType = None
    if salary_and_jType == None:
        try:
            salary_and_jType = soup.find('div',id="icl-u-xs-block jobsearch-ReqAndQualSection-item--title").text.replace("\n", "").strip()
        except:
            salary_and_jType = None
    #TODO get benefits from its designated section
    
    
    try:
        sal_guide_items = []
        items = soup.find('ul',class_='css-1lyr5hv eu4oa1w0')
        for i in items:
            sal_guide_items.append(i.text)
    except:
        sal_guide_items = None
        
        
    try:
        salfromsection = soup.find('span',class_='icl-u-xs-mr--xs').text
    except:
        salfromsection = None
        
        
    try:
        job_type_items = []
        job_type_from_section = soup.find('div',class_='jobsearch-JobDescriptionSection-sectionItem').next_sibling.children
        for i in job_type_from_section:
            if i.text == 'Job Type':
                continue
            else:
                job_type_items.append(i.text)
    except:
        job_type_items = None
        
    
    try:
        requirements = soup.find(class_="icl-u-xs-block jobsearch-ReqAndQualSection-item--title").text.replace("\n", "").strip()      

    except:
        requirements = None
        
        
    try:
        description = soup.find(id="jobDescriptionText").text.replace('\n', '')
    except:
        description = None
        
        
    # A nifty little workaround for evading detection.
    time.sleep(.5+random()*3)
    #TODO assess h2 tags commonalities to determine if these section descriptions are from Indeed or are at least of only a few variations.
        #you could then distinguish the description into sections and conduct NLP etc each.
    raw_desc_soup = soup
    return salary_and_jType, sal_guide_items, salfromsection, job_type_items, requirements, description, raw_desc_soup

In [20]:
#TODO condense these with lists, particularly fields that have .text.strip()
def get_features(post):
    """parses search results and extracts basic job feature values,
        then combines this with output of 'get_desc_features' function.

    Args:
        post (string): response for each post in search results page

    Returns:
        dict: single-feature deep dictionary of features (dictionary keys) and their values (dictionary values)
    """
    datapoint_dict = {}

    title = post.find('h2',
              attrs={'class': lambda e: e.startswith('jobTitle') if e else False}).text.replace('new', '')

    company = post.find('span', 'companyName').text.strip()
    try:
        rating = post.find('span', 'ratingNumber').text
    except:
        rating = None

    location = post.find('div', 'companyLocation').text.strip()
    postDate = post.find('span', 'date').text
    extractDate = datetime.today().strftime('%Y-%m-%d')
    summary = post.find('div', 'job-snippet').text.strip().replace('\n', ' ')
    url = 'https://www.indeed.com'+ post.find('a', href = re.compile(r'[/]([a-z]|[A-Z])\w+')).attrs['href']

    try:
        estimated_salary = post.find('span','estimated-salary').text.strip()
    except:
        estimated_salary = None
    try:
        salary = post.find('div','metadata salary-snippet-container').text.strip()
    except:
        salary = None


        
            
    salary_and_jType, sal_guide_items, salfromsection, job_type_items, requirements, description, raw_desc_soup = get_desc_features(url)
    datapoint_dict = {
                        'title':title,
                        'company':company,
                        'rating':rating,
                        'location':location,
                        'salary':salary,
                        'estimated_salary':estimated_salary,
                        'postDate':postDate,
                        'extractDate':extractDate,
                        'summary':summary,
                        'url':url,
                        'salary_and_jType':salary_and_jType,
                        'sal_guide_items':sal_guide_items,
                        'salfromsection':salfromsection,
                        'job_type_items':job_type_items,
                        'requirements':requirements,
                        'description':description,
                        'raw_desc_soup':raw_desc_soup}
    if len(datapoint_dict) > 0:
        return datapoint_dict
    else:
        pass

In [28]:
def main(position, location):
    """[Conducts the web scraping process]
    Args:
        position ([string]): [job position for indeed.com query]
        position ([string]): [job location for indeed.com query]
        
        Returns:
        [csv]: [scraped data]
    """
    data = pd.DataFrame()
    
    # extract the job data
    while True:
        response = requests.get(get_URL(position, location))
        soup = BeautifulSoup(response.text, 'html.parser')
        searchResults = soup.find('div', id='mosaic-provider-jobcards')
        refinedsearchResults = searchResults.find_all('div', attrs={'class': lambda e: e.startswith('cardOutline') if e else False})
        

        raw_posts = []
        for post in refinedsearchResults:
            raw_posts.append(post)
        
        n = 0
        for post in raw_posts:
            datapoint = get_features(post)
            data = data.append(datapoint, ignore_index=True)
        # Again, a nifty little workaround for evading detection.
            n+=1
            print(n)
            
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except:
            break

    name = position.replace(' ','_')
    loc = location.replace(' ','_')
    day = date.today()
    # save the job data
    data.to_csv(f'../app/data/scraped_{name}_{loc}.csv', index=False)
    return raw_posts

In [44]:
#state_names = [ "alabama", "arkansas",  "arizona",  "colorado", "connecticut", "delaware",  "georgia", "iowa", "idaho", "illinois", "indiana", "kansas", "kentucky", "louisiana", "massachusetts", "maryland", "maine", "michigan", "minnesota", "missouri", "mississippi", "montana", "north carolina", "north dakota", "nebraska", "new hampshire", "new jersey", "new mexico", "nevada",  "ohio", "oklahoma", "oregon", "pennsylvania", "rhode island", "south carolina", "south dakota", "tennessee", "utah", "virginia",  "vermont",  "wisconsin", "west virginia", "wyoming"]


state_names = [ "pennsylvania", "rhode island", "south carolina", "south dakota", "tennessee", "utah", "virginia",  "vermont",  "wisconsin", "west virginia", "wyoming"]

for state in state_names:
    position = 'data scientist'
    location = state
    print(state)
    data = main(position,location )

pennsylvania


AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
position = 'data scientist'
location = 'california'
main(position,location )

In [ ]:
position = 'data scientist'
location = 'remote'
main(position,location )

In [ ]:
position = 'data scientist'
location = 'new york'
main(position,location )

In [ ]:
position = 'data scientist'
location = 'texas'
main(position,location )

In [ ]:
position = 'data scientist'
location = 'washington'
main(position,location )

In [41]:
position = 'data scientist'
location = 'florida'
main(position,location )

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
position = 'data scientist'
location = 'massachusetts'
main(position,location )

In [ ]:
position = 'data scientist'
location = 'oregon'
main(position,location )

The  below is used for various adjustments to my webscraping process.

In [43]:
pd.read_csv(f'../app/data/scraped_data_scientist_oregon.csv')

,company,description,estimated_salary,extractDate,job_type_items,location,postDate,rating,raw_desc_soup,requirements,sal_guide_items,salary,salary_and_jType,salfromsection,summary,title,url
0,Recruiting From Scratch,Who is Recruiting from Scratch: Recruiting fro...,NaN,2022-06-21,NaN,"Remote in Portland, OR 97035",PostedToday,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,"$160,000 - $200,000 a year","$160,000 - $200,000 a year","$160,000 - $200,000 a year",5+ years experience in data engineering or dat...,Data Scientist,https://www.indeed.com/rc/clk?jk=0921a823554b3...
1,Fisher Investments,Are you highly analytical and looking for an o...,Estimated $56.7K - $71.9K a year,2022-06-21,NaN,"Portland, OR 97204+1 location",PostedToday,3.5,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,"['', 'Not provided by employer', ""$56.7K - $71...",NaN,NaN,NaN,Be a part of our data modernization efforts th...,Data Science Associate,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...


### Concatenating Old Data With New

In [46]:
state_names = [ "alabama", "arkansas",  "arizona", "california", "colorado", "connecticut", "delaware", "florida", "georgia", "remote", "iowa", "idaho", "illinois", "indiana", "kansas", "kentucky", "louisiana", "massachusetts", "maryland", "maine", "michigan", "minnesota", "missouri", "mississippi", "montana", "north carolina", "north dakota", "nebraska", "new hampshire", "new jersey", "new mexico", "nevada", "new york", "ohio", "oklahoma", "oregon", "pennsylvania", "rhode island", "south carolina", "south dakota", "tennessee", "texas", "utah", "virginia",  "vermont", "washington", "wisconsin", "west virginia", "wyoming"]

lst = []
records = 0
for i in state_names:
    i = i.replace(' ','_')
    try:
        location = pd.read_csv(f'../app/data/scraped_data_scientist_{i}.csv')
        print(f'{i} has {len(location)} records')
        records += len(location)
        lst.append(location)
    except:
        continue

print(records)

todays_scrape = pd.concat(lst)
## unblock if you miss too many days
#total = todays_scrape.to_csv('../app/data/total.csv', index= False)

total = pd.read_csv(f'../app/data/total.csv')

lst.append(total)

total = pd.concat(lst)
total.to_csv('../app/data/total.csv', index= False)
total

alabama has 7 records
arkansas has 1 records
arizona has 10 records
colorado has 10 records
connecticut has 30 records
delaware has 7 records
georgia has 45 records
iowa has 3 records
idaho has 1 records
illinois has 30 records
indiana has 10 records
kansas has 6 records
kentucky has 2 records
louisiana has 5 records
massachusetts has 210 records
maryland has 12 records
maine has 1 records
michigan has 7 records
minnesota has 9 records
missouri has 3 records
mississippi has 1 records
montana has 1 records
north_carolina has 13 records
north_dakota has 1 records
nebraska has 1 records
new_hampshire has 1 records
new_jersey has 15 records
new_mexico has 1 records
nevada has 1 records
ohio has 10 records
oklahoma has 1 records
oregon has 2 records
457


,company,description,estimated_salary,extractDate,job_type_items,location,postDate,rating,raw_desc_soup,requirements,sal_guide_items,salary,salary_and_jType,salfromsection,summary,title,url
0,Recruiting From Scratch,Who is Recruiting from Scratch: Recruiting fro...,NaN,2022-06-21,NaN,"Remote in Huntsville, AL",PostedToday,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,"$160,000 - $200,000 a year","$160,000 - $200,000 a year","$160,000 - $200,000 a year",5+ years experience in data engineering or dat...,Data Scientist,https://www.indeed.com/rc/clk?jk=977d1ccb6d1a1...
1,Food Management Search,SR Financial Data Analyst- Fully remote - ANY ...,NaN,2022-06-21,NaN,Remote in United States,PostedToday,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,NaN,NaN,NaN,Lead various allocation data analysis projects...,SR Financial Data Analyst- Fully remote,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,Gregor Diagnostics,Role: Senior Data ScientistAbout Gregor Diagno...,NaN,2022-06-21,NaN,Remote in United States,PostedPosted 1 day ago,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,NaN,Full-time,NaN,The Senior Data Scientist will work closely la...,Senior Data Scientist,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
3,SynergisticIT,"At SynergisticIT, we aim to bring aboard IT ...",Estimated $77.7K - $98.3K a year,2022-06-21,NaN,Alabama,PostedPosted 1 day ago,4.2,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,"['', 'Not provided by employer', ""$77.7K - $98...",NaN,"Full-time, Contract",NaN,"Collaborate with dynamic teams of engineers, d...",Entry Level Data Scientist,https://www.indeed.com/rc/clk?jk=57d47b0524890...
4,9Rooftops,WE ARE HIRING IN MULTIPLE LOCATIONS ACROSS THE...,Estimated $109K - $138K a year,2022-06-21,NaN,"Birmingham, AL 35242",PostedPosted 1 day ago,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,"['', 'Not provided by employer', ""$109K - $138...",NaN,NaN,NaN,O Communication skills to ask questions of cli...,Statistician/Data Scientist,https://www.indeed.com/rc/clk?jk=e23726e90096e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,Sanofi,Overview At Sanofi Large Molecule Research Pla...,NaN,2022-06-21,NaN,"Cambridge, MA 02238 (Mid-Cambridge area)+1 loc...",PostedToday,4.1,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,NaN,NaN,NaN,Track record of applying machine learning/ dee...,Principal Data Scientist Machine Learning for ...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
373,PAREXEL,NaN,NaN,2022-06-21,NaN,Massachusetts,PostedToday,3.6,NaN,NaN,NaN,NaN,NaN,NaN,As a Statistical Programmer you will support v...,Statistical Programmer I (Poland),https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
374,NATIONAL GRID CO USA (NE POWER),NaN,NaN,2022-06-21,NaN,"Waltham, MA 02454",PostedToday,3.7,"<html lang=""en"">\n<head>\n<title>hCaptcha solv...",NaN,NaN,NaN,NaN,NaN,"Employ sophisticated analytics, machine learni...","Lead Data Scientist, Data Science",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
375,PAREXEL,NaN,NaN,2022-06-21,NaN,Massachusetts,PostedToday,3.6,NaN,NaN,NaN,NaN,NaN,NaN,The Senior Statistical Programmer provides tec...,Senior Statistical Programmer FSP (Poland),https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...


The below are bits of code I employ if something goes wrong with the webscraping process.

In [ ]:
"""#fix old imports

data['extractDate']= pd.to_datetime(data['extractDate'])

def pDate(row):
    from datetime import datetime, date, timedelta

    #days_ago = row['dateposted']
    delta = timedelta(0)
    try:
        return row['extractDate'] - delta
    except:
        return row

data['extractDate'] = data.apply( lambda row : pDate(row), axis = 1)
data['extractDate'] = data['extractDate'].astype(str)
#data.to_csv('../app/data/scraped_data_scientist_remote_2022-04-14.csv', index= False)
data.extractDate.unique()"""

In [ ]:
"""# codescraps in case they change the html and break my parsers

searchResults = soup.find('div', id='mosaic-provider-jobcards')
refinedsearchResults = searchResults.find_all('div', attrs={'class': lambda e: e.startswith('cardOutline') if e else False})
len(refinedsearchResults)
z = searchResults.children
lst = []
for i in z:
    lst.append(i)

x = lst[0]

#checking again
soup = BeautifulSoup(response.text, 'html.parser')
#searchResults = soup.find('div', id='mosaic-provider-jobcards')
refinedsearchResults = soup.find_all('div', attrs={'class': lambda e: e.startswith('cardOutline') if e else False})
        

raw_posts = []
for post in refinedsearchResults:
        raw_posts.append(post)
        n = 0

z = raw_posts[0]
url = z.find('a', href=True)
url


z.find('a', href = re.compile(r'[/]([a-z]|[A-Z])\w+')).attrs['href']
z

postDate = z.find('span', 'date').text
extractDate = datetime.today().strftime('%Y-%m-%d')
summary = z.find('div', 'job-snippet').text.strip().replace('\n', ' ')

summary

company_name = z.find('a', attrs={'class':'turnstileLink companyOverviewLink'}).text.strip()
company_name

job_title = z.find('a', attrs={'class':'jcs-JobTitle'}).text.strip()
job_title

"""